In [2]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records
import models
models_path = models.__path__[0]
import os

fix_block_number = 41354245

cmf_param = {
    'block_number': fix_block_number,
    'chain_id': 137,
}

context, _model_loader = create_cmf(cmf_param)

In [3]:
from models.credmark.protocols.ichi.vaults import ICHI_POLYGON_COINS, ICHI_POLYGON_VAULTS

In [4]:
for coin in ICHI_POLYGON_COINS:
    tok = Token(coin).as_erc20(set_loaded=True)
    print((tok.symbol, tok.name, tok.decimals))

('WMATIC', 'Wrapped Matic', 18)
('LINK', 'ChainLink Token', 18)
('GOVI', 'GOVI (PoS)', 18)
('BAL', 'Balancer (PoS)', 18)
('GHST', 'Aavegotchi GHST Token (PoS)', 18)
('USDC', 'USD Coin (PoS)', 6)
('CRV', 'CRV (PoS)', 18)
('WETH', 'Wrapped Ether', 18)
('SUSHI', 'SushiToken (PoS)', 18)
('oneBTC', 'oneBTC', 18)
('ICHI', 'ICHI', 18)
('DPI', 'DefiPulse Index (PoS)', 18)
('WBTC', '(PoS) Wrapped BTC', 8)
('TRADE', 'Polytrade (PoS)', 18)


In [4]:
from models.tmp_abi_lookup import ICHI_VAULT, ICHI_VAULT_FACTORY

In [11]:
if False:
    vault_factory = Contract('0x2d2c72C4dC71AA32D64e5142e336741131A73fc0').set_abi(ICHI_VAULT_FACTORY, set_loaded=True)
    vault_created = pd.DataFrame(vault_factory.fetch_events(vault_factory.events.ICHIVaultCreated, from_block=25_697_834, by_range=10_000))
    vault_created.to_pickle(os.path.join(models_path, '..', 'tmp', 'vault_created.pkl'))
else:
    vault_created = pd.read_pickle(os.path.join(models_path, '..', 'tmp', 'vault_created.pkl'))

# vault_created.ichiVault.isin(['0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81'])
# '0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81' in vault_created.ichiVault.to_list()

# '0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81' in ICHI_POLYGON_VAULTS
assert len(set(ICHI_POLYGON_VAULTS)) == len(set(vault_created.ichiVault))

# context.run_model('token.deployment', {'address': '0x2d2c72C4dC71AA32D64e5142e336741131A73fc0'})

In [16]:
for n_vault, vault in enumerate(ICHI_POLYGON_VAULTS):
    vault = Contract(vault).set_abi(abi=ICHI_VAULT, set_loaded=True)
    token0 = Token(vault.functions.token0().call()).as_erc20(set_loaded=True)
    token1 = Token(vault.functions.token1().call()).as_erc20(set_loaded=True)

    print((n_vault, vault,
           vault.functions.owner().call(), vault.functions.pool().call(),
           token0.symbol,  token1.symbol,
           vault.functions.allowToken0().call(), vault.functions.allowToken1().call(),
           vault.functions.totalSupply().call() / (10**vault.functions.decimals().call())))
    
    # print((vault, vault.functions.pool().call()))

(0, Contract(address='0x9ff3c1390300918b40714fd464a39699ddd9fe00'), '0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0x86f1d8390222A3691C28938eC7404A1661E618e0', 'WMATIC', 'WETH', True, False, 203.4025762204604)
(1, Contract(address='0x692437de2cae5addd26ccf6650cad722d914d974'), '0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0x3e31AB7f37c048FC6574189135D108df80F0ea26', 'LINK', 'WETH', True, False, 123.25275146959906)
(2, Contract(address='0x70aa3c8e256c859e52c0b8c263f763d9051b95e1'), '0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0x713A96308455046a61af91bC220eC83fdc920977', 'ICHI', 'GOVI', False, True, 300002.8217032606)
(3, Contract(address='0xf461063819ffbc6e22704ade1861b0df3bac9585'), '0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0x4fe1269a585B141F11C3E144158f9f8823c7C0e7', 'WETH', 'BAL', False, True, 16119.092595672058)
(4, Contract(address='0xf3de925524ce6bba606107cfcb2a7a6259cd01ea'), '0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0x8f16A8E864162ec84a2906646E08a561b5A7f72d', 'GHST

In [13]:
# allowToken0, allowToken1, balanceOf, getTotalAmounts, totalSupply, token0, token1, symbol, name41354245
# getBasePosition, getLimitPosition, hysteresis, 
# owner, ichiVaultFactory, pool, tickSpacing, fee

In [ ]:
# vault operations
# withdraw, transfer
# deposit, transfer
# rebalance

# 0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7

In [25]:
deposit_block = 41028728

context.web3.eth.default_block = deposit_block

vault_dpi = Token('0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7').set_abi(abi=ICHI_VAULT, set_loaded=True)

depositor = '0x3fabe6f79f5119336886bbbdfc0b0853dc831919'

vault_dpi.scaled(vault_dpi.balance_of(Address(depositor).checksum))


278.10168836934116

In [27]:
context.web3.eth.default_block = fix_block_number

pd.DataFrame(vault_dpi.fetch_events(vault_dpi.events.Withdraw, from_block=deposit_block, by_range=1000))

""


In [10]:
vault

(19, 19, 19)